In [1]:
# set of stop words from nltk
import nltk
import os
from os import listdir
#from nltk.corpus import stopwords
#stopwordsset = set(stopwords.words('english'))

## Choosing set of StopWords

In [2]:
stopwordsset = {'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 
 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",
 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself',
 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th'}

## Splitting the Document Files into Train and Test Files

In [3]:
# split the data into train and test
from sklearn.model_selection import train_test_split
path = r'C:\Users\MrLather\Docsspace\20_newsgroups'
pathOfAllFiles = []
Features = []
for f in listdir(path):
    Features.append(f)
Y = []
c = 0
for r,d,f in os.walk(path):
    for file in f:
        p = r + '\\' + file
        pathOfAllFiles.append(p)
        Y.append(Features[c - 1])
    c += 1
print(len(Y), len(pathOfAllFiles))
doc_train, doc_test, Y_train, Y_test = train_test_split(pathOfAllFiles, Y, random_state=0, test_size=0.25)

19892 19892


## Selecting meaningfull words to Train and Test the Data

In [4]:
from nltk.stem import WordNetLemmatizer 
import string 
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

# Funtion to return lemmatized word
def lemmatize(tup):
    word = tup[0]
    tag = tup[1]
    wnl = WordNetLemmatizer()
    if tag.startswith("NN"):
        return wnl.lemmatize(word, pos='v')
    elif tag.startswith('VB'):
        return wnl.lemmatize(word, pos='v')
    elif tag.startswith('RB'):
        return wnl.lemmatize(word, pos='a')
    elif tag.startswith('JJ'):
        return wnl.lemmatize(word, pos='a')
    else:
        return word

# Funtion to return only meaningfull words for training
def tokenize(wordlist):
    word = []
    for w in wordlist:
        # removing punctuations from the word
        w = ''.join(c for c in w if c not in string.punctuation)
        # selecting only words of length > 2 and containing only alphabets
        if w.isalpha() and w not in stopwordsset and len(w) > 2: 
            word.append(lemmatize(pos_tag(word_tokenize(w.lower()))[0]))
    return word

## Storing the frequency of selected words in a Dictionary

In [5]:
wordlist = {} # to store all words with their frequency
Xtraindata = {} # to store all words frequency document wise
for filepath in doc_train:
    # read the document ans store in a string
    stri = open(filepath, "r").read()
    # storing all meaningfull words in a list
    words = tokenize(stri.split(' '))
    Xtraindata[filepath] = words
    for w in words:
        if w in wordlist:
            wordlist[w] = wordlist[w] + 1
        else:
            wordlist[w] = 1
#print(wordlist)

## Sort the Dictionary to select most frequently occuring words as features

In [6]:
import operator
feature = {} # to store the features names with its index
# li contains sorted list of tuples (key,value) from dictionary
li = sorted(wordlist.items(), key = operator.itemgetter(1))
l = len(li)
n = 4000
for i in range(n):
    feature[li[l - i - 1][0]] = i

## Making Data to Train and Test

In [7]:
Xtrain = []
for filepath in doc_train:
    temp = [0 for _ in range(n)]
    for w in Xtraindata[filepath]:
        if w in feature:
            temp[feature[w]] += 1
    Xtrain.append(temp)

In [8]:
print(sum([sum(Xtrain[i]) for i in range(len(Xtrain))]))

984818


In [9]:
Xtest = []
for filepath in doc_test:
    temp = [0 for _ in range(4000)]
    stri = open(filepath, "r").read()
    words = tokenize(stri.split(' '))
    for w in words:
        if w in feature:
            temp[feature[w]] += 1
    Xtest.append(temp)

## Using InBuilt NaiveBayes

In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

In [11]:
clf = MultinomialNB(alpha=1)
clf.fit(Xtrain,Y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [12]:
#y_pred = clf.predict(X_test)
print("Testing Score of InBuilt NaiveBayes : ", clf.score(Xtest, Y_test))

Testing Score of InBuilt NaiveBayes :  0.6412628192238086


In [13]:
print("Training Score of InBuilt NaiveBayes : ", clf.score(Xtrain, Y_train))

Training Score of InBuilt NaiveBayes :  0.6848314230176286


In [14]:
import numpy as np
Xtrain = np.array(Xtrain)
Y_train = np.array(Y_train)
Xtest = np.array(Xtest)
Y_test = np.array(Y_test)

## Using our own NaiveBayes Classifier

In [15]:
# This function will fit the training data and return a dictionary storing the various counts
def fit(X_train, Y_train):
    dic = {}
    classes = set(Y_train)
    dic["total_data_points"] = len(Y_train)
    for curr_class in classes:
        dic[curr_class] = {}
        features = X_train.shape[1]
        curr_class_rows = (Y_train == curr_class) # gives us true false values array acc to the condition only if Ytrain is numpy array
        X_train_current = X_train[curr_class_rows]
        Y_train_current = Y_train[curr_class_rows]
        dic[curr_class]["total"] = len(Y_train_current)
        for j in range(features):
            dic[curr_class][j] = {}
            posval = set(X_train[:, j])
            for k in posval:
                dic[curr_class][j][k] = (X_train_current[:, j] == k).sum() # gives us sum of all true values satisfying the condition
    return dic

# Return the log values of probabilities Using the Bayes Theorem
def probability(dic, xi, curr_class):
    #curr_class_prob = (dic[curr_class]["total"] / dic["total_data_points"])
    # the above probability calculations are now converted to log values to avoid very smaller results
    curr_class_prob = np.log(dic[curr_class]["total"]) - np.log(dic["total_data_points"])
    #print(dic[curr_class])
    for j in dic[curr_class]:
        if j != "total":
            num = dic[curr_class][j][xi[j]] + 1
            den = dic[curr_class]["total"] + len(dic[curr_class][j])
            curr_class_prob = curr_class_prob + np.log(num) - np.log(den)
    return curr_class_prob

# This will Predict results for single row of testing data and return the resultant class
def predSingle(dic, xi):
    classes = dic.keys()
    #print(classes)
    firstRun = True
    for curr_class in classes:
        if curr_class != "total_data_points":
            prob_curr_class = probability(dic, xi, curr_class)
            if (firstRun or prob_curr_class > maxp):
                maxp = prob_curr_class
                maxclass = curr_class
                firstRun = False
    return maxclass

# This function will predict the results and return them as an array
def predict(X_test, result):
    y_pred = []
    for i in range(len(X_test)):
        y_pred.append(predSingle(result, X_test[i]))
    return y_pred

# This function will return the score by comparing the actual results with the predicting results
def score(y_pred, Y_test):
    score = 0
    for i in range(len(y_pred)):
        if y_pred[i] == Y_test[i]:
            score = score + 1
    score = score / len(y_pred)
    return score    

## Making continuous data labelled to fit into our algorithm

In [16]:
def label(column, middle):
    # we divide the column values into 4 ranges i.e by dividing the line with 3 points midddle point is the mean
    first_point = 0.5 * middle
    third_point = 1.5 * middle
    for i in range(len(column)):
        if column[i] < first_point:
            column[i] = 0
        elif column[i] < middle:
            column[i] = 1
        elif column[i] < third_point:
            column[i] = 2
        else:
            column[i] = 3
    return column

In [17]:
for i in range(len(Xtrain[0])):
    middle = (Xtrain[:,i].mean() + Xtest[:,i].mean()) / 2
    Xtrain[:,i] = label(Xtrain[:,i], middle)
    Xtest[:,i] = label(Xtest[:,i], middle)

In [18]:
result = fit(np.array(Xtrain), np.array(Y_train))

In [19]:
y_pred = predict(np.array(Xtest), result)

In [20]:
print("Testing Score of our algorithm is : ", score(y_pred, Y_test))

Testing Score of our algorithm is :  0.6155238286748441


## Printing Classification Report and Confusion Matrix

In [21]:
from sklearn.metrics import classification_report as cr, confusion_matrix as cm
print("Classification_Report : -")
print(cr(Y_test, y_pred))
print("##############################")
print()
print("Confusion_Matrix : -")
print(cm(Y_test, y_pred))

Classification_Report : -
                          precision    recall  f1-score   support

             alt.atheism       0.79      0.77      0.78       233
           comp.graphics       0.71      0.44      0.55       257
 comp.os.ms-windows.misc       0.75      0.65      0.70       247
comp.sys.ibm.pc.hardware       0.78      0.64      0.70       240
   comp.sys.mac.hardware       0.79      0.73      0.76       235
          comp.windows.x       0.54      0.56      0.55       242
            misc.forsale       0.48      0.83      0.61       258
               rec.autos       0.74      0.59      0.66       278
         rec.motorcycles       0.90      0.74      0.81       257
      rec.sport.baseball       0.95      0.71      0.81       240
        rec.sport.hockey       0.75      0.57      0.65       234
               sci.crypt       0.90      0.81      0.85       247
         sci.electronics       0.71      0.56      0.62       264
                 sci.med       0.93      0.59    

Rough Code

In [22]:
#Dummy Trial Code
#f = open("ReadDoc/untitled.txt", "r") # to read the file
#dir(f) # to check all methods avail with f
#string = f.read() # to read line from the file
#from nltk.tokenize import word_tokenize 
#word_tokens = word_tokenize(string) 
#for i in word_tokens:
    #print(i, end = "  ")
#import os
#from os import listdir
#directorylist = []
#path = r'C:\Users\MrLather\Docsspace\20_newsgroups'
#c = 0
# os.walk iterate on all the folders , subfolders inside the path specified including the current folder (20newgroup)
#     so if there are 20 folder inside 20news then this loop will run 21 times including the 20newsfolder
# r will give names(path) of all 21 folders
# d will give a list of names of all folders inside the current r so we get 21 lists(some may be empty)
#     where first list will denote the folders inside 20news itself i.e those 20 folders
# f will give a list of all the files names that are directly inside the current r not in its subfolder
#     so first f contains 0 files in this case while second f contain all files in the first subfolder of 20news folder
'''for r,d,f in os.walk(path):
    c += 1
    if(c == 1):
        print(f)
print(c)'''

'''for f in listdir(path):
    print(f)'''

'''directorylist = []
for r,d,f in os.walk(path):
    c += 1
    if(c == 2):
        t = '\\'
        p = r + t + f[0]
        fil = open(p, "r")
        #print(tokenize(fil.read().split(' ')))
        text_lines = fil.readlines()
        text_lines = remove_metadata(text_lines)
        for i in text_lines:
            print(i.strip(" "),end = "")'''

'directorylist = []\nfor r,d,f in os.walk(path):\n    c += 1\n    if(c == 2):\n        t = \'\\\'\n        p = r + t + f[0]\n        fil = open(p, "r")\n        #print(tokenize(fil.read().split(\' \')))\n        text_lines = fil.readlines()\n        text_lines = remove_metadata(text_lines)\n        for i in text_lines:\n            print(i.strip(" "),end = "")'